In [1]:
# Let's import all required libraries
import selenium                                  #library that is used to work with selenium
from selenium import webdriver                   #importing webdriver module from selenium to open automated chrome window
import pandas as pd                              #to create DataFrame
import numpy as np
from selenium.webdriver.common.by import By      #importing inbuilt class By
import time
import re
# Importing required Exceptions which needs to be handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
'''
Q1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up amazon.in website on automated chrome window
url = 'https://www.amazon.in/'
driver.get(url)

# initialize the variable to store user input
user_input = input("Enter the product you want to search for:\n")

# finding web element for search
search_p = driver.find_element(By.XPATH,"//input[@id='twotabsearchtextbox']")
search_p.send_keys(user_input)

# clicking on search button
search_btn = driver.find_element(By.XPATH, "//input[@id='nav-search-submit-button']")
search_btn.click()
time.sleep(1)

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/863563914.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


Enter the product you want to search for:
shoes


In [3]:
'''
Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and
save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products
available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange",
"Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then
replace it by “-“.
'''
# fetching url to open each product page for upto 3 pages only
product_url = []
start = 1
while True:
    url = driver.find_elements(By.XPATH, '//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-2"]/a')
    for i in url:
        product_url.append(i.get_attribute('href'))     # to retrieve any link, we use get_attribute
    # handling exception if next page is not available
    start+=1
    if start <= 3:
        try:
            next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
            next_button.click()
            time.sleep(3)

        except NoSuchElementException:
            print(f"Exception Raised: Next Page not found.\nOnly {start} page/s were extracted.")
            break
    else: break

# creating an empty list
brand_name = []
product_name = []
product_price = []
return_exchange = []
expected_delivery = []
p_availability = []

delay = 20
url = 0      # using url variable we will append '-' for any missing value.

for i in product_url:
    driver.get(i)          # taking each url one by one as an input
    time.sleep(5)
    
    url += 1
    
    try:
        
        try:
            
            # Check if the price is available or it asks to select size else select size from the given select menu or if it is unavailable
            text1 = driver.find_element(By.XPATH, '//div[@id="apex_offerDisplay_desktop"]').text

            if len(text1) != 0:

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//a[@id="bylineInfo"]')))
                brand = driver.find_element(By.XPATH, '//a[@id="bylineInfo"]')
                brand_name.append(brand.text.replace('Visit the ','').replace(' Store','').replace('Brand: ',''))

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//h1[@id="title"]')))
                name = driver.find_element(By.XPATH, '//h1[@id="title"]')
                product_name.append(name.text)

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')))
                price = driver.find_element(By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')
                product_price.append(price.text.replace("\n","."))

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')))
                rne = driver.find_element(By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')
                return_exchange.append(rne.text)

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')))
                delivery = driver.find_element(By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')
                expected_delivery.append(delivery.text.replace(". Details\n", " ").replace(". Details\nSelect delivery location","").replace(" Select delivery location",""))

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="availability"]')))
                availability = driver.find_element(By.XPATH, '//div[@id="deliveryBlock_feature_div"]')
                p_availability.append(availability.text.replace("Details\n","").replace("Details",""))
                
        except NoSuchElementException:
            
            # Check if it asks to select size else select size from the given select menu or if it is unavailable
            text = driver.find_element(By.XPATH, '//div[@class="a-section a-spacing-small a-text-center"]//span[1]').text
            if text == "To buy, select Size":

                # clicking on size button if present
                button = driver.find_element(By.XPATH, '//*[@id="native_dropdown_selected_size_name"]')
                button.click()
                time.sleep(1)

                # clicking on any available size from list present
                button = driver.find_element(By.XPATH, '//*[@id="native_dropdown_selected_size_name"]//option[3]')
                button.click()
                time.sleep(4)

                text2 = driver.find_element(By.XPATH, '//div[@id="apex_offerDisplay_desktop"]').text

                if len(text2) != 0:

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//a[@id="bylineInfo"]')))
                    brand = driver.find_element(By.XPATH, '//a[@id="bylineInfo"]')
                    brand_name.append(brand.text.replace('Visit the ','').replace(' Store','').replace('Brand: ',''))

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//h1[@id="title"]')))
                    name = driver.find_element(By.XPATH, '//h1[@id="title"]')
                    product_name.append(name.text)

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')))
                    price = driver.find_element(By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')
                    product_price.append(price.text.replace("\n","."))

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')))
                    rne = driver.find_element(By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')
                    return_exchange.append(rne.text)

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')))
                    delivery = driver.find_element(By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')
                    expected_delivery.append(delivery.text.replace(". Details\n", " ").replace(". Details\nSelect delivery location","").replace(" Select delivery location",""))

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="availability"]')))
                    availability = driver.find_element(By.XPATH, '//div[@id="deliveryBlock_feature_div"]')
                    p_availability.append(availability.text.replace("Details\n","").replace("Details",""))

                else:

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//a[@id="bylineInfo"]')))
                    brand = driver.find_element(By.XPATH, '//a[@id="bylineInfo"]')
                    brand_name.append(brand.text.replace('Visit the ','').replace(' Store','').replace('Brand: ',''))

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//h1[@id="title"]')))
                    name = driver.find_element(By.XPATH, '//h1[@id="title"]')
                    product_name.append(name.text)

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')))
                    price = driver.find_element(By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')
                    product_price.append("-")

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')))
                    rne = driver.find_element(By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')
                    return_exchange.append("-")

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')))
                    delivery = driver.find_element(By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')
                    expected_delivery.append("-")

                    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="availability"]')))
                    availability = driver.find_element(By.XPATH, '//div[@id="deliveryBlock_feature_div"]')
                    p_availability.append("-")

            elif text == "Currently unavailable.":

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//a[@id="bylineInfo"]')))
                brand = driver.find_element(By.XPATH, '//a[@id="bylineInfo"]')
                brand_name.append(brand.text.replace('Visit the ','').replace(' Store','').replace('Brand: ',''))

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//h1[@id="title"]')))
                name = driver.find_element(By.XPATH, '//h1[@id="title"]')
                product_name.append(name.text)

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')))
                price = driver.find_element(By.XPATH, '//div[@class="a-section a-spacing-none a-padding-none"]/div[2]')
                product_price.append("-")

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')))
                rne = driver.find_element(By.XPATH, '//span[@class="a-declarative"]/div[@class="a-section a-spacing-none icon-content"]')
                return_exchange.append("-")

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')))
                delivery = driver.find_element(By.XPATH, '//div[@id="apex_offerDisplay_single_desktop"]')
                expected_delivery.append("-")

                WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@id="availability"]')))
                availability = driver.find_element(By.XPATH, '//div[@id="deliveryBlock_feature_div"]')
                p_availability.append("-")
        
    except :
        
        if len(brand_name) != url:
            brand_name.append('-')
        if len(product_name) != url:
            product_name.append('-') 
        if len(product_price) != url:
            product_price.append('-') 
        if len(return_exchange) != url:
            return_exchange.append('-') 
        if len(expected_delivery) != url:
            expected_delivery.append('-') 
        if len(p_availability) != url:
            p_availability.append('-') 
# print(len(brand_name),len(product_name),len(product_price), len(return_exchange), len(expected_delivery), len(p_availability))

In [8]:
#creating a data frame
df = pd.DataFrame({'Brand Name':brand_name, 'Name of the Product':product_name, 'Price':product_price, 'Return/Exchange':return_exchange, 'Expected Delivery':expected_delivery, 'Availability':p_availability, 'Product URL':product_url})

# saving the dataframe into csv
df.to_csv('file1.csv')

df

,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,ASIAN,"ASIAN Men's Thar-04 Sports Running,Walking & G...",₹474.00,30 Days Returns & Exchange,"FREE delivery Sunday, 27 November on first ord...","FREE delivery Sunday, 27 November on first ord...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,ASIAN,"ASIAN Men's Newton-09 Sports Running,Walking &...",₹578.00,30 Days Returns & Exchange,"FREE delivery Saturday, 26 November Or fastest...","FREE delivery Saturday, 26 November. Or fastes...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Bourge,Bourge Mens Loire-z-168 Running Shoe,₹702.30,30 Days Returns & Exchange,"FREE delivery Sunday, 27 November Or fastest d...","FREE delivery Sunday, 27 November. Or fastest ...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Avant,AVANT Men's Nitro Running and Gym Shoes - Grey...,₹649.00,30 Days Returns & Exchange,"FREE delivery Tuesday, 29 November","FREE delivery Tuesday, 29 November.",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Campus,Campus Men's OXYFIT (N) Walking Shoe,₹679.00,30 Days Returns & Exchange,"FREE delivery Sunday, 27 November Or fastest d...","FREE delivery Sunday, 27 November. Or fastest ...",https://www.amazon.in/Campus-OXYFIT-Walking-Sh...
...,...,...,...,...,...,...,...
194,ASIAN,"ASIAN Men's Carbon-02 Sports Running,Walking &...","₹1,139.00",30 Days Returns & Exchange,"FREE delivery Sunday, 27 November Or fastest d...","FREE delivery Sunday, 27 November. Or fastest ...",https://www.amazon.in/ASIAN-Carbon-02-Running-...
195,ASIAN,"ASIAN Men's Cosco-13 Sports Running,Walking & ...",₹497.00,30 Days Returns & Exchange,"FREE delivery Sunday, 27 November on first ord...","FREE delivery Sunday, 27 November on first ord...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
196,Kraasa,Kraasa Sports Shoes for Men | Walking Shoes | ...,₹599.00,30 Days Returns & Exchange,"FREE delivery Sunday, 27 November Or fastest d...","FREE delivery Sunday, 27 November. Or fastest ...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
197,Generic,Men's Black Mesh Sports Shoes,₹599.00,30 Days Returns & Exchange,"₹59 delivery Monday, 28 November","₹59 delivery Monday, 28 November.",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [25]:
'''
Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for
keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

images = {'fruits':[], 'cars':[], 'Machine Learning':[], 'Guitar':[], 'Cakes':[]}
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

delay = 15

# storing the images url for each search keywords.
for key in keywords:
    
    # Opening up images.google.com website on automated chrome window
    url = 'https://images.google.com/'
    driver.get(url)
    time.sleep(1)

    # finding web element for search
    search_p = driver.find_element(By.XPATH,"//input[@class='gLFyf']")
    search_p.send_keys(key)

    # clicking on search button
    search_btn = driver.find_element(By.XPATH, "//span[@class='z1asCe MZy1Rb']")
    search_btn.click()
    time.sleep(1)
    
    for i in range(1,11):
        driver.find_element(By.XPATH, '//*[@id="islrg"]/div[1]/div[%s]'%i).click()
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//img[@class="n3VNCb KAlRDb"]')))
        im = driver.find_element(By.XPATH, '//img[@class="n3VNCb KAlRDb"]')
        images[key].append(im.get_attribute('src'))
    time.sleep(2)

# saving the images to folder for each search keywords
import requests

for key in images:
    imag = 0
    
    for i in images[key]:
        
        imag += 1
        x = key
        response = requests.get(i)
        file = open(r'C:\Users\katiy\Downloads\Web Scraping Assignment 3\{}\ '.format(x)+str(imag)+'.jpg','wb')
        file.write(response.content)
        
print("10 images for each keyword is saved in the respective folder of each keyword")

C:\Users\katiy\AppData\Local\Temp/ipykernel_1364/807223903.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


10 images for each keyword is saved in the respective folder of each keyword


In [13]:
'''
Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape
following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”,
“Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”,
“Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.
'''

# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up flipkart.com website on automated chrome window
url = 'https://www.flipkart.com/'
driver.get(url)

# clicking on CLOSE button of login popup
close = driver.find_element(By.XPATH, '//button[@class="_2KpZ6l _2doB4z"]')
close.click()
time.sleep(1)

# finding web element for search for products, brands and more bar
search_p = driver.find_element(By.CLASS_NAME,"_3704LK")
search_p.send_keys("smartphone")

# clicking on search button
search_btn = driver.find_element(By.XPATH, '//button[@class="L0Z3Pu"]')
search_btn.click()
time.sleep(7)


# initializing required empty list
smartphone_brand = []
smartphone_model = []
color = []
ram = []
storage = []
p_camera = []
s_camera = []
disp_size = []
bat_cap = []
price = []
product_url = []

# Let's extract all urls for the each displayed item on 1st search page
product_url_tags = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for i in product_url_tags:
    product_url.append(i.get_attribute('href'))

url = 0      # using url variable we will append '-' for any missing value.
delay = 15

# Now iterate through each URL link and extract data required from each page.
for urls in product_url:
    driver.get(urls)
    time.sleep(3)
    
    url += 1
    
    try:
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a')))
        brand_tag = driver.find_element(By.XPATH,'//*[@id="container"]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a')
        smartphone_brand.append(brand_tag.text)
        
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="_30jeq3 _16Jk6d"]')))
        price_tag = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
        price.append(price_tag.text)
        time.sleep(2)
        
    except (NoSuchElementException, TimeoutException):
        smartphone_brand.append('-')
        price.append('-')
    
    # Find the Read More button and clicking it
    driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']").click()
    time.sleep(1)
    
    # Let's store all table rows as directly accessing required data is difficult
    smartphone = driver.find_elements(By.XPATH,"//tr[@class='_1s_Smc row']")
    time.sleep(3)
    
    # Now iterate through each row stored in smartphone variable
    for element in smartphone:
        search_text = element.find_element(By.XPATH,'td[1]').text       # Store first column text for each row which we will use for our if condition below
        
        # Let's extract for Brand name and phone name
        if search_text == 'Model Name':
            smartphone_model.append(element.find_element(By.XPATH,"td[2]").text)
            
        # Let's extract for color
        if search_text == 'Color':
            color.append(element.find_element(By.XPATH,"td[2]").text)
            
        # Let's extract for RAM
        if search_text == 'RAM':
            ram.append(element.find_element(By.XPATH,"td[2]").text)
            
        # Let's extract for storage(ROM)
        if search_text == 'Internal Storage':
            storage.append(element.find_element(By.XPATH,"td[2]").text)
            
        # Let's extract for Primary Camera
        if search_text == 'Primary Camera':
            p_camera.append(element.find_element(By.XPATH,"td[2]").text)
            
        # Let's extract for Secondary Camera
        if search_text == 'Secondary Camera':
            s_camera.append(element.find_element(By.XPATH,"td[2]").text)
        
        # Let's extract for Display Size
        if search_text == 'Display Size':
            disp_size.append(element.find_element(By.XPATH,"td[2]").text)
        
        # Let's extract for Display Size
        if search_text == 'Battery Capacity':
            bat_cap.append(element.find_element(By.XPATH,"td[2]").text)
    
    # Check if all list are of equal length for a given page and append '-' for any missing values
    if len(smartphone_model) != url:
        smartphone_model.append('-')
    if len(color) != url:
        color.append('-') 
    if len(ram) != url:
        ram.append('-') 
    if len(storage) != url:
        storage.append('-') 
    if len(p_camera) != url:
        p_camera.append('-') 
    if len(s_camera) != url:
        s_camera.append('-') 
    if len(disp_size) != url:
        disp_size.append('-') 
    if len(bat_cap) != url:
        bat_cap.append('-')

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/4294950782.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


In [14]:
smartphone_df=pd.DataFrame({"Brand Name":smartphone_brand,"Smartphone name":smartphone_model,"Colour":color,"RAM":ram,"Storage(ROM)":storage,"Primary Camera":p_camera,"Secondary Camera":s_camera,"Display Size":disp_size,"Battery Capacity":bat_cap,"Price":price,"Product URL":product_url}).style

# saving the dataframe into csv
df.to_csv('file2.csv')

smartphone_df

,Brand Name,Smartphone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,APPLE Mobiles,iPhone 13,Starlight,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),-,"₹65,999",https://www.flipkart.com/apple-iphone-13-starlight-128-gb/p/itmc9604f122ae7f?pid=MOBG6VF5ADKHKXFX&lid=LSTMOBG6VF5ADKHKXFXQGX7PK&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBG6VF5ADKHKXFX.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
1,APPLE Mobiles,iPhone 13,Midnight,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),-,"₹65,999",https://www.flipkart.com/apple-iphone-13-midnight-128-gb/p/itmca361aab1c5b0?pid=MOBG6VF5Q82T3XRS&lid=LSTMOBG6VF5Q82T3XRSOXJLM9&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBG6VF5Q82T3XRS.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
2,APPLE Mobiles,iPhone 13,Green,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),-,"₹65,999",https://www.flipkart.com/apple-iphone-13-green-128-gb/p/itm18a55937b2607?pid=MOBGC9VGSU9DWGJZ&lid=LSTMOBGC9VGSU9DWGJZ0Q1T8E&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBGC9VGSU9DWGJZ.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
3,APPLE Mobiles,iPhone 13,Blue,-,128 GB,12MP + 12MP,12MP Front Camera,15.49 cm (6.1 inch),-,"₹65,999",https://www.flipkart.com/apple-iphone-13-blue-128-gb/p/itm6c601e0a58b3c?pid=MOBG6VF5SMXPNQHG&lid=LSTMOBG6VF5SMXPNQHGL5FN51&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_4&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBG6VF5SMXPNQHG.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
4,POCO Mobiles,C31,Royal Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹9,499",https://www.flipkart.com/poco-c31-royal-blue-64-gb/p/itm19effae969b86?pid=MOBG73E7GKQK4KZP&lid=LSTMOBG73E7GKQK4KZPR5ICMK&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_5&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBG73E7GKQK4KZP.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
5,vivo Mobiles,T1 5G,Starlight Black,6 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.71 cm (6.58 inch),5000 mAh,"₹16,990",https://www.flipkart.com/vivo-t1-5g-starlight-black-128-gb/p/itm594222523bd8f?pid=MOBGB9TYF7P7RNYX&lid=LSTMOBGB9TYF7P7RNYX5GJVDV&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_6&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBGB9TYF7P7RNYX.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
6,vivo Mobiles,T1 5G,Rainbow Fantasy,4 GB,128 GB,50MP + 2MP + 2MP,16MP Front Camera,16.71 cm (6.58 inch),5000 mAh,"₹15,990",https://www.flipkart.com/vivo-t1-5g-rainbow-fantasy-128-gb/p/itm594222523bd8f?pid=MOBGB9TYGW5NGXVH&lid=LSTMOBGB9TYGW5NGXVHWMF5TV&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_7&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBGB9TYGW5NGXVH.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
7,POCO Mobiles,C31,Shadow Gray,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,"₹9,499",https://www.flipkart.com/poco-c31-shadow-gray-64-gb/p/itm162375acb8370?pid=MOBG73E7UBFXXMCH&lid=LSTMOBG73E7UBFXXMCHSM0ZN5&marketplace=FLIPKART&q=smartphone&store=tyy%2F4io&srno=s_1_8&otracker=search&otracker1=search&fm=organic&iid=1f4d5ac7-dbe7-4e5d-a30a-548dd44f3194.MOBG73E7UBFXXMCH.SEARCH&ppt=hp&ppn=homepage&ssid=93jpqrxtio0000001669309082956&qH=1036ba3c4ed2c021
8,APPLE Mo

In [15]:
'''
Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up maps.google.com website on automated chrome window
url = 'https://maps.google.com/'
driver.get(url)

key = input("Enter the city for which you want to find the coordinates:\n")

# finding web element for search for products, brands and more bar
search_p = driver.find_element(By.XPATH,'//input[@id="searchboxinput"]')
search_p.send_keys(key)

# clicking on search button
search_btn = driver.find_element(By.XPATH, '//button[@id="searchbox-searchbutton"]')
search_btn.click()
time.sleep(5)

url = driver.current_url        # fetch the url for current page

# initialize variables for string between which text needs to be extracted
sub1 = "@"
sub2 = ",1"
 
# getting index of substrings
index1 = url.index(sub1)
index2 = url.index(sub2)

# initialize empty string variable
res = ''
# getting elements in between '@' and ',1'
for idx in range(index1 + len(sub1), index2):
    res = res + url[idx]

# printing result
print(f"The geospatial coordinates for {key} city are: ({res})")

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/1080669967.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


Enter the city for which you want to find the coordinates:
delhi
The geospatial coordinates for delhi city are: (28.6437954,76.8130369)


In [16]:
'''
6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up trak.in website on automated chrome window
url = 'https://trak.in/'
driver.get(url)

# clicking on funding deals
search_btn = driver.find_element(By.XPATH, "//a[normalize-space()='Funding Deals']")
search_btn.click()
time.sleep(2)

class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_column_info(self,month):
        page = self.driver.find_elements(By.XPATH,'//div[@class="bs-vc-wrapper wpb_wrapper"]')
        time.sleep(3)

        for element in page:
            search_text = element.find_element(By.XPATH,'(//h2[normalize-space()="{}"])[1]'.format(month))
            xpath = search_text.get_attribute('id')
            xpath = xpath.split("-n")[0]
        column_info = []
        column_tags = self.driver.find_elements(By.XPATH,'//div[@id="{}_wrapper"]//div[@class="dataTables_scrollHead"]//th'.format(xpath))
        for i in column_tags:
            column_info.append(i.text)
        return column_info
    
    def get_results(self, month, index = None):
        
        page = self.driver.find_elements(By.XPATH,'//div[@class="bs-vc-wrapper wpb_wrapper"]')
        time.sleep(3)

        for element in page:
            search_text = element.find_element(By.XPATH,'(//h2[normalize-space()="{}"])[1]'.format(month))
            xpath = search_text.get_attribute('id')
            xpath = xpath.split("-n")[0]
            
        columns = self.get_column_info(month)
        data = {}
        time.sleep(2)
        if index == None:
            a = ""
        else:
            a = '[{}]'.format(index)
        elements = self.driver.find_elements(By.XPATH,'(//div[@id="{}_wrapper"]//div[@class="dataTables_scrollBody"]//tbody/tr){}'.format(xpath, a))
        for element in elements:
            current_index = index if index != None else elements.index(element) + 1
            parsed_data = []
            for column in columns:
                value = element.find_element(By.XPATH,'(//div[@id="{}_wrapper"]//div[@class="dataTables_scrollBody"]//tbody/tr)[{}]/td[{}]'.format(xpath, current_index, columns.index(column) + 1)).text
                parsed_data.append(value)
            data.update({current_index: parsed_data})
        data = pd.DataFrame(data.values())
        # data.set_axis(columns, axis=1, inplace=True)
        data.columns = columns
        return data


if "__main__" == __name__:
    
    table = Table(driver)
    results_jan_21 = table.get_results("January, 2021")
    results_feb_21 = table.get_results("February, 2021")
    results_mar_21 = table.get_results("March, 2021")

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/2390980852.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


In [17]:
results_jan_21

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,2,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,3,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,5,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,6,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,7,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,8,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"


In [18]:
results_feb_21

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
1,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
2,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"
3,4,17/02/2021,Zolve,FinTech,Global Neobank Venture,Mumbai,Accel Partners and Lightspeed Venture Partners,Seed,"1,50,00,000"
4,5,15/02/2021,KreditBee,Finance,Digital lending platform,Bengaluru,Azim Premji’s PremjiInvest and South Korea’s M...,Series C,"75,000,000"
5,6,12/02/2021,Pepperfry,E-commerce,Multi-brand furniture brand,Mumbai,InnoVen Capital,Debt Financing,"4,773,958"
6,7,12/02/2021,Grofers,E-Commerce,Online supermarket,Gurgaon,SoftBank Vision Fund (SVF),Unspecified,"55,000,000"
7,8,09/02/2021,Nothing,Technology,Consumer Technology Venture,London,GV,Series A,"15,000,000"
8,9,09/02/2021,SplashLearn,EdTech,Game-based learning programme,Gurgaon,Owl Ventures,Series C,"18,000,000"


In [19]:
results_mar_21

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
5,6,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"
6,7,26/03/2021,DotPe,FinTech,Commerce and payments platform to offline ente...,Gurgaon,PayU,Series A,"27,500,000"


In [20]:
'''
Q7. Write a program to scrap all the available details of best gaming laptops from digit.in.
'''

# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up digit.in website on automated chrome window
url = 'https://www.digit.in/'
driver.get(url)
time.sleep(1)

# clicking on funding deals
search_btn = driver.find_element(By.XPATH, "//li/a[contains(text(),'Best Gaming Laptops')]")
search_btn.click()
time.sleep(2)

# initializing required empty list
laptop_name = []
processor = []
display = []
os_ = []
memory = []
graphics_processor = []
body = []

# for storing laptops name:
laptop_name_tags = driver.find_elements(By.XPATH, '//div[@class="left_side"]//h3')
for i in laptop_name_tags:
    laptop_name.append(i.text)

# for storing processor's name:
processor_tags = driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[@class = "Spcs-details"]//tbody/tr[3]/td[3]')
for i in processor_tags:
    processor.append(i.text)

# for storing display specs:
display_tags = driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[@class = "Spcs-details"]//tbody/tr[4]/td[3]')
for i in display_tags:
    display.append(i.text)

# for storing operating system detail:
os__tags = driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[@class = "Spcs-details"]//tbody/tr[5]/td[3]')
for i in os__tags:
    os_.append(i.text)

# for storing RAM and storage details:
memory_tags = driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[@class = "Spcs-details"]//tbody/tr[6]/td[3]')
for i in memory_tags:
    memory.append(i.text)

# for storing graphics card details:
graphics_processor_tags = driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[@class = "Spcs-details"]//tbody/tr[7]/td[3]')
for i in graphics_processor_tags:
    graphics_processor.append(i.text)

# for storing dimesion and weight
body_tags = driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[@class = "Spcs-details"]//tbody/tr[8]/td[3]')
for i in body_tags:
    body.append(i.text)


df = pd.DataFrame({'Laptop Name':laptop_name, 'Processor':processor, 'Display':display, 'Operating System':os_, 'RAM & Storage':memory, 'Graphics Card':graphics_processor, 'Dimension & Weight':body})
df

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/1873794437.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


,Laptop Name,Processor,Display,Operating System,RAM & Storage,Graphics Card,Dimension & Weight
0,MSI Titan GT77-12UHS,12th Gen Intel Core i9-12900HX 16 core processor,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight
1,Alienware X17 R2,12th Gen Intel Core i9-12900H 14 core processo...,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension & 2.96 kg weight
2,Acer Predator Triton 500 SE PT516-52s,12th Gen Intel Core i7-12700H 14 core processo...,16″ (2560 x 1600) screen,Windows 11,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,262 x 359 x 20 mm dimension & 2.4 kg weight
3,Omen By HP (16-B1371TX),12th Gen Intel Core i7-12700H 14 core processo...,16.1″ (2560 x 1440) screen,Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight
4,Acer Predator Helios 300 AN515-45,AMD Ryzen 9-5900HX 8 core processor,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight
5,MSI Delta 15 (A5EFK-083IN),AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight
6,Omen By HP (16-C0141AX),AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16.1″ (2560 x 1440) screen,Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight
7,Lenovo Legion 5i Pro (82RF00MGIN),12th Gen Intel Core i7-12700H 14 core processo...,"16″ (2560 x 1600) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension & 2.49 kg weight
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,AMD Ryzen R7-5800H 8 core processor with 4.40 ...,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,"15.6″ screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight


In [21]:
'''
8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up digit.in website on automated chrome window
url = 'https://www.forbes.com/'
driver.get(url)
time.sleep(1)

# clicking on menu button
click_btn = driver.find_element(By.XPATH, '//div[@aria-label="Close Navigation Menu"]')
click_btn.click()
time.sleep(1)

# clicking on Billionaires menu
click_btn = driver.find_element(By.XPATH, '//li[1]//div[1]//span[1]//*[name()="svg"]')
click_btn.click()
time.sleep(1)

# clicking on World's Billionaires
click_btn = driver.find_element(By.XPATH, '//a[normalize-space()="World\'s Billionaires"]')
click_btn.click()
time.sleep(2)

# clicking on first row to close it
click_btn = driver.find_element(By.XPATH, '(//div[@role="cell"])[2]')
click_btn.click()
time.sleep(2)

class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_column_info(self):
        column_info = []
        column_tags = self.driver.find_elements(By.XPATH,'//div[@class="header-row"]//div[@class="header-name"]')
        for i in column_tags:
            column_info.append(i.text)
        return column_info
    
    def get_results(self):
        
        page = driver.find_elements(By.XPATH,'//div[@class="bs-vc-wrapper wpb_wrapper"]')
        time.sleep(3)

        columns = ['RANK', 'NAME', 'NET WORTH', 'AGE', 'COUNTRY / TERRITORY', 'SOURCE', 'INDUSTRY']
        data = []
        tab = []
        time.sleep(5)
        tables = driver.find_elements(By.XPATH,'//div[@class="table-body"]/div[@class="table-row-group"]')
        for table in tables:
            current_index_table = tables.index(table) + 1
            elements = driver.find_elements(By.XPATH,'//div[@class="table-body"]/div[@class="table-row-group"][{}]//div[@class="table-row "]'.format(current_index_table))
            for element in elements:
                current_index = elements.index(element) + 1
                parsed_data = []
                for column in columns:
                    value = element.find_element(By.XPATH,'//div[@class="table-body"]/div[@class="table-row-group"][{}]//div[@class="table-row "][{}]/div[{}]'.format(current_index_table,current_index, columns.index(column) + 1)).text
                    parsed_data.append(value)
                data.append(parsed_data)
            tab.append(data)
        df = pd.DataFrame(tab[0])
        #df.set_axis(columns, axis=1, inplace=True)
        df.columns = columns
        df
        return df


if "__main__" == __name__:
    
    table = Table(driver)
    top_billionaires = table.get_results()

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/2668417955.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


In [22]:
top_billionaires

,RANK,NAME,NET WORTH,AGE,COUNTRY / TERRITORY,SOURCE,INDUSTRY
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [4]:
'''
Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up amazon.in website on automated chrome window
url = 'https://www.youtube.com/watch?v=XtttYHljxyc'
driver.get(url)
time.sleep(3)

driver.execute_script("window.scrollBy(0,500)")
time.sleep(2)

# Get scroll height
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")

    # Wait to load page
    time.sleep(4)

    # Calculate new scroll height and compare with last scroll height
    comments = driver.find_elements(By.XPATH, '//ytd-comment-thread-renderer')
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if len(comments)>500:
        break
    
    elif new_height == last_height:
        break
    last_height = new_height

commenter = []
comment_time = []
comment_content = []
upvotes = []
a = 0
for i in comments:
    a += 1
    
    commenter_tag = i.find_element(By.XPATH, '//ytd-comment-thread-renderer[{}]//h3/a/span'.format(a)).text
    if commenter_tag == "":
        commenter.append(i.find_element(By.XPATH, '//ytd-comment-thread-renderer[{}]//yt-formatted-string[@id="text"]'.format(a)).text)
    else:
        commenter.append(commenter_tag)
    
    try:
        upvotes_tags = i.find_element(By.XPATH, '//ytd-comment-thread-renderer[{}]//span[contains(@id,"vote-count-middle")]'.format(a))
        if upvotes_tags.text != '':
            upvotes.append(upvotes_tags.text)
        else:
            upvotes.append('-')
            
    except:
        upvotes.append("-")
    comment_time.append(i.find_element(By.XPATH, '//ytd-comment-thread-renderer[{}]//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]'.format(a)).text)
    comment_content.append(i.find_element(By.XPATH,'//ytd-comment-thread-renderer[{}]//div[@id="comment-content"]'.format(a)).text.replace("\n", " "))

df = pd.DataFrame({"Commenter":commenter, 'Comment':comment_content, 'Upvote':upvotes, 'Comment Time':comment_time})
df

C:\Users\katiy\AppData\Local\Temp/ipykernel_1364/532931731.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


,Commenter,Comment,Upvote,Comment Time
0,SSC Utkarsh,SSC GD (Constable) ब्रह्मास्त्र Free Special ...,261,8 days ago
1,Vinod Meena ___________,कोन कोन मानता है कुमार गौरव सर INDIA के No.1 T...,1.8K,9 days ago
2,ITz__ Naitik,Guru Ji No Words For you ️ Love You ️ Sir,2,11 hours ago
3,aspiRANT vloGS,Thankuu soo much sir for this session,-,8 hours ago
4,@VikramKarsena,"आपकी क्लास सभी एग्जामो के लिए महत्वपूर्ण है, च...",-,9 hours ago
...,...,...,...,...
515,Lokesh Yadav,board ki pdf bhi provide krwa diya kro plz,-,7 days ago
516,success academy,Current session kara dijiye sir,-,8 days ago
517,Smile Datrata,Sir Content bahut accha hai. Thank you so........,-,6 days ago
518,SRG Specials,CGL ka current affairs krwa do sir plz,-,7 days ago


In [56]:
'''
Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location.
You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price,
dorms from price, facilities and property description.
'''
# Connect to web driver
driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")

# Maximize the automated chrome window
driver.maximize_window()

# Opening up maps.google.com website on automated chrome window
url = 'https://www.hostelworld.com/'
driver.get(url)

# clicking in search bar to send keys
click_btn = driver.find_element(By.XPATH, '//input[@id="location-text-input-field"]')
click_btn.click()
time.sleep(2)

# finding web element for search for products, brands and more bar
search_p = driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
search_p.send_keys("London")
time.sleep(5)

# clicking on London under city
click_btn = driver.find_element(By.XPATH, '//div[@class="search-form-area"]//li[2]')
click_btn.click()
time.sleep(5)

# clicking on Let's Go to search for the available hotels
click_btn = driver.find_element(By.XPATH, '//button[@id="search-button"]')
click_btn.click()
time.sleep(5)

# clicking on filter
click_btn = driver.find_element(By.XPATH, '//button[@id="listoptions-filter"]')
click_btn.click()
time.sleep(1)

# clicking on Property Type
click_btn = driver.find_element(By.XPATH, '//div[@class="filters"]//a/div[text()="Property Type"]')
click_btn.click()
time.sleep(1)

# clicking on Hostels
click_btn = driver.find_element(By.XPATH, '//div[@id="property-type-filter"]//label[text()="Hostels"]')
click_btn.click()
time.sleep(2)

# create a list for storing all result pages link
page_urls = []
page_urls.append(driver.current_url)

while True:
    try:
        # clicking on next page
        click_btn = driver.find_element(By.XPATH, '//div[@class="pagination-item pagination-next"]')
        click_btn.click()
        time.sleep(3)
        page_urls.append(driver.current_url)
    except:
        break

hostel_name = []
distance_from_city_centre = []
ratings = []
total_reviews = []
overall_reviews = []
privates_from_price = []
dorms_from_price = []
facilities = []
property_description = []
hostel_url = []

# iterate through each result page
for url in page_urls:
    driver.get(url)          # taking each url one by one as an input
    time.sleep(3)
    
    hostel = driver.find_elements(By.XPATH, '//div[@class="page-inner"]/div/div/div[@class="property-card"]//div[@class="title-row"]/h2/a')
    for i in hostel:
        hostel_name.append(i.text)

    distance_from_city_centre = []
    distance = driver.find_elements(By.XPATH, '//div[@class="page-inner"]/div/div/div[@class="property-card"]//div[@class="title-row"]/div/a/span[1]')
    for i in distance:
        distance_from_city_centre.append(i.text.split("Hostel - ")[1])

    overall_reviews = []
    o_r = driver.find_elements(By.XPATH, '//div[@class="page-inner"]/div/div/div[@class="property-card"]//div[@class="info"]//div[@class="keyword"]')
    for i in o_r:
        overall_reviews.append(i.text)

    total_reviews = []
    review = driver.find_elements(By.XPATH, '//div[@class="page-inner"]/div/div/div[@class="property-card"]//div[@class="info"]//div[@class="reviews"]')
    for i in review:
        total_reviews.append(i.text)

    privates_from_price = []
    pp = driver.find_elements(By.XPATH, '//div[@class="page-inner"]/div/div/div[@class="property-card"]//div[@class="prices-col"]/a/div[1]')
    for i in pp:
        privates_from_price.append(i.text)

    # Cleaning results to get price offered
    for i in range(len(privates_from_price)):
        up_to_word = "Rs"
        rx_to_last = r'^.*{}'.format(re.escape(up_to_word))
        privates_from_price[i] = re.sub(rx_to_last, '', privates_from_price[i], flags=re.DOTALL)

    dp = driver.find_elements(By.XPATH, '//div[@class="page-inner"]/div/div/div[@class="property-card"]//div[@class="prices-col"]/a/div[2]')
    for i in dp:
        dorms_from_price.append(i.text)

    # Cleaning results to get price offered
    for i in range(len(dorms_from_price)):
        up_to_word = "Rs"
        rx_to_last = r'^.*{}'.format(re.escape(up_to_word))
        dorms_from_price[i] = re.sub(rx_to_last, '', dorms_from_price[i], flags=re.DOTALL)
        
    # on each search page get links for each hostel
    hostel_url_tags = driver.find_elements(By.XPATH,'//div[@class="page-inner"]/div/div/div[@class="property-card"]//h2/a')
    for i in hostel_url_tags:
        hostel_url.append(i.get_attribute('href'))

    delay = 15
    # Now iterate through each URL link and extract data required from each page.
    for urls in hostel_url:
        driver.get(urls)
        time.sleep(3)

        try:
            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class = "flex-20"]//div[contains(@aria-label,"Rating:")]')))
            ratings_tag = driver.find_element(By.XPATH,'//div[@class = "flex-20"]//div[contains(@aria-label,"Rating:")]')
            ratings.append(ratings_tag.text)

            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//ul[@class="facility-sections"]')))
            facilities_tag = driver.find_element(By.XPATH,'//ul[@class="facility-sections"]')
            facilities.append(facilities_tag.text.replace("\n"," "))
            time.sleep(2)

            WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//div[@class="flex-80"]/div[@class="description-container"]/div[@class="collapsible"]/div[2]')))
            property_description_tag = driver.find_element(By.XPATH,'//div[@class="flex-80"]/div[@class="description-container"]/div[@class="collapsible"]/div[2]')
            property_description.append(property_description_tag.text.replace("\n"," "))
            time.sleep(2)

        except (NoSuchElementException, TimeoutException):
            pass

df = pd.DataFrame({'Hostel Name':hostel_name, 'Distance from City Centre':distance_from_city_centre, 'Ratings':ratings, 'Total reviews':total_reviews, 'Overall Review':overall_reviews, 'Privates from Price (Rs)':privates_from_price,'Dorms from price (Rs)':dorms_from_price,'Facilities':facilities,'Property Description':property_description})
df

C:\Users\katiy\AppData\Local\Temp/ipykernel_25596/1141669732.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\katiy\Downloads\chromedriver.exe")


,Hostel Name,Distance from City Centre,Ratings,Total reviews,Overall Review,Privates from Price (Rs),Dorms from price (Rs),Facilities,Property Description
0,Hostel One Notting Hill,5.5km from city centre,9.4,1594 Total Reviews,Superb,17386,7889,Free Linen Included Free WiFi Free Internet Ac...,The perfect place for solo travelers to connec...
1,St Christopher's Village,1.8km from city centre,8.1,11697 Total Reviews,Fabulous,No Privates Available,4942,Free Linen Included Free City Maps Free WiFi F...,COVID 19 Policy Update. In response to Coronav...
2,Generator London,3km from city centre,7.7,7259 Total Reviews,Very Good,No Privates Available,5731,Free Linen Included Free City Maps Free WiFi F...,Generator London is a design hotel-hostel loca...
3,Safestay London Kensington Holland Park,5.9km from city centre,6.8,1377 Total Reviews,Good,No Privates Available,4578,Free Linen Included Free WiFi General Security...,Safestay Holland Park Safestay Holland Park i...
4,St Christopher's Camden,4.3km from city centre,8.3,3899 Total Reviews,Fabulous,No Privates Available,4746,Free Linen Included Free City Maps Free WiFi F...,COVID 19 Policy Update. In response to Coronav...
5,NX London Hostel,6.1km from city centre,7.6,1345 Total Reviews,Very Good,No Privates Available,5634,Free Free Breakfast Linen Included Free Parkin...,Welcome to NX London Hostel! A Friendly backp...
6,St Christopher's Hammersmith,7.5km from city centre,7.8,4156 Total Reviews,Very Good,No Privates Available,4225,Free Linen Included Free City Maps Free WiFi F...,"In response to Coronavirus (COVID-19), additio..."
7,Smart Hyde Park View Hostel,5km from city centre,7.9,4635 Total Reviews,Very Good,31172,No Dorms Available,Free Linen Included Free City Maps Free WiFi G...,Our Hyde Park View Hostel is in a superb locat...
8,Saint James Backpackers,5.5km from city centre,7.3,1846 Total Reviews,Very Good,13006,5445,Free Free Breakfast Linen Included Free City M...,We are located in central London - just 5 minu...
9,St Christopher's Greenwich,7.6km from city centre,6.7,3278 Total Reviews,Good,No Privates Available,3769,Free Linen Included Free City Maps Free WiFi F...,COVID 19 Policy Update. In response to Coronav...
